In [2]:
import os
import glob
import pandas as pd
import xarray as xr
import numpy as np

### Open and read the netcdf file 

In [13]:
path = '/home/datawork-cersat-public/project/sarwave/data/products/tests2/slc/iw/l1c/2017/288/S1B_IW_XSP__1SDV_20171015T184419_20171015T184446_007844_00DD96_2573_B07.SAFE/l1c-s1b-iw3-xsp-1sdv-20171015t184419-20171015t184444-007844-00dd96-b07.nc'

ds = xr.open_dataset(path, group ='intraburst')
ds

<xarray.Dataset> Size: 15MB
Dimensions:                   (tile_line: 9, tile_sample: 4, pol: 2,
                               bt_thresh: 5, c_sample: 2, c_line: 2,
                               freq_sample: 506, freq_line: 50, k_gp: 4,
                               phi_hf: 5, lambda_range_max_macs: 11, 2tau: 1)
Coordinates:
    line                      (tile_line) int16 18B ...
    sample                    (tile_line, tile_sample) int16 72B ...
    longitude                 (tile_line, tile_sample) float32 144B ...
    latitude                  (tile_line, tile_sample) float32 144B ...
  * pol                       (pol) <U2 16B 'VH' 'VV'
  * bt_thresh                 (bt_thresh) int16 10B 5 50 100 150 200
    k_rg                      (tile_line, tile_sample, freq_sample) float32 73kB ...
    k_az                      (freq_line) float32 200B ...
  * k_gp                      (k_gp) int16 8B 1 2 3 4
  * phi_hf                    (phi_hf) int16 10B 1 2 3 4 5
  * lambda_range_max_macs     (lambda_range_max_macs) float32 44B 25.0 ... 275.0
    crs                       int16 2B ...
Dimensions without coordinates: tile_line, tile_sample, c_sample, c_line,
                                freq_sample, freq_line, 2tau
Data variables: (12/116)
    incidence                 (tile_line, tile_sample) float32 144B ...
    ground_heading            (tile_line, tile_sample) float32 144B ...
    sensing_time              (tile_line, tile_sample) datetime64[ns] 288B ...
    tile_width_sample         (tile_line, tile_sample) int16 72B ...
    tile_width_line           (tile_line, tile_sample) int16 72B ...
    sigma0                    (tile_line, tile_sample, pol) float32 288B ...
    ...                        ...
    vabr                      (tile_line, tile_sample) float32 144B ...
    uubr                      (tile_line, tile_sample) float32 144B ...
    vubr                      (tile_line, tile_sample) float32 144B ...
    mssu                      (tile_line, tile_sample) float32 144B ...
    mssc                      (tile_line, tile_sample) float32 144B ...
    mssd                      (tile_line, tile_sample) float32 144B ...
Attributes: (12/24)
    orbit_pass:                    Ascending
    azimuth_time_interval:         0.002055556299999998
    short_name:                    SENTINEL1_DS:S1B_IW_SLC__1SDV_20171015T184...
    required_tile_width_line:      17500
    product:                       SLC
    swath:                         IW
    ...                            ...
    platform:                      SENTINEL-1B
    required_tile_width_sample:    17500
    history:                       digital_number: measurement/s1b-iw3-slc-v*...
    name:                          SENTINEL1_DS:/home/datawork-cersat-public/...
    azimuth_steering_rate:         1.397440818
    ipf:                           2.84

In [12]:
selected_vars = ['sigma0_filt','incidence','nesz', 'ground_heading','land_flag',
                 'uwnd', 'vwnd','U10', 'V10', 'macs_Im', 'macs_Re', 'normalized_variance_filt']


if not set(selected_vars).issubset(ds.keys()):
            print(f'All Variables not found in {path}')
            #return None

ds = ds.drop_vars('crs')
ds = ds[selected_vars]
display(ds)

All Variables not found in /home/datawork-cersat-public/project/sarwave/data/products/tests2/slc/iw/l1c/2017/263/S1B_IW_XSP__1SDV_20170920T102318_20170920T102345_007474_00D31F_10FD_B07.SAFE/l1c-s1b-iw3-xsp-1sdv-20170920t102318-20170920t102343-007474-00d31f-b07.nc


KeyError: 'U10'

In [91]:
# Supposons que 'pol', 'tile_line', et 'tile_sample' soient les dimensions pertinentes
tile_line = ds.sizes['tile_line']; tile_sample = ds.sizes['tile_sample']
pol_values = ds['pol'].values  # ['VH', 'VV']

display(tile_line, tile_sample, pol_values)
# Créer une grille pour les dimensions
tile_line_indices, tile_sample_indices, pol_indices = np.meshgrid(np.arange(tile_line), np.arange(tile_sample), np.arange(len(pol_values)), indexing='ij')

# Réorganiser les indices pour qu'ils soient sous forme linéaire
pol_indices = pol_indices.ravel()

# Obtenir les valeurs de 'pol' associées aux indices
expanded_pol = pol_values[pol_indices]

# Convertir le dataset en DataFrame
df_res = ds.drop_dims(['lambda_range_max_macs']).squeeze().to_dataframe().reset_index(drop=True).drop(columns=['spatial_ref', 'sample', 'line'], errors='ignore')

# Ajouter la colonne 'pola' avec les valeurs correctement alignées
df_res['pola'] = expanded_pol
df_res

9

5

array(['VH', 'VV'], dtype='<U2')

,sigma0_filt,incidence,nesz,ground_heading,land_flag,uwnd,vwnd,U10,V10,normalized_variance_filt,longitude,latitude,pola
0,NaN,36.645969,0.002884,-11.656672,True,-3.123083,7.790683,-2.632508,7.035500,NaN,-6.448264,61.827137,VH
1,NaN,36.645969,0.002547,-11.656672,True,-3.123083,7.790683,-2.632508,7.035500,NaN,-6.448264,61.827137,VV
2,0.000150,37.728565,0.002138,-11.337094,False,-2.708674,7.691035,-2.268131,7.284603,1.191188,-6.122775,61.857773,VH
3,0.014344,37.728565,0.001876,-11.337094,False,-2.708674,7.691035,-2.268131,7.284603,1.188372,-6.122775,61.857773,VV
4,0.000132,38.787601,0.001923,-11.057025,False,-2.619153,7.559871,-2.190142,7.034854,1.086010,-5.796837,61.887619,VH
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0.050575,38.775448,0.001664,-11.282922,False,-8.300492,8.016449,-9.440802,9.261126,1.231618,-6.344853,63.194866,VV
86,0.001234,39.810261,0.002005,-10.975616,False,-8.340130,7.723401,-8.938865,8.594180,1.099593,-6.003569,63.224537,VH
87,0.047706,39.810261,0.001723,-10.975616,False,-8.340130,7.723401,-8.938865,8.594180,1.215896,-6.003569,63.224537,VV
88,0.001088,40.821430,0.002622,-10.667827,False,-8.327045,7.528501,-8.020763,7.828473,1.081216,-5.661509,63.253391,VH


In [87]:
for _lambda_max in ds['lambda_range_max_macs'].values:
    df = ds[['macs_Im', 'macs_Re']].sel(lambda_range_max_macs = _lambda_max).to_dataframe().reset_index(drop=True).drop(columns=['lambda_range_max_macs', 'spatial_ref', 'sample', 'line', 'pol'], errors='ignore')
    df = df.rename(columns={'macs_Im': f'macs_Im_lambda_max={_lambda_max}',
                            'macs_Re': f'macs_Re_lambda_max={_lambda_max}'})
    df['pola']=expanded_pol
    df_res = df_res.merge(df, on=['longitude', 'latitude','pola'])

#df_res2 = pd.merge(df, df_res1, on=['longitude', 'latitude','pola'], how='inner')
display(df_res)

,sigma0_filt,incidence,nesz,ground_heading,land_flag,uwnd,vwnd,U10,V10,normalized_variance_filt,...,macs_Im_lambda_max=175.0,macs_Re_lambda_max=175.0,macs_Im_lambda_max=200.0,macs_Re_lambda_max=200.0,macs_Im_lambda_max=225.0,macs_Re_lambda_max=225.0,macs_Im_lambda_max=250.0,macs_Re_lambda_max=250.0,macs_Im_lambda_max=275.0,macs_Re_lambda_max=275.0
0,NaN,36.645969,0.002884,-11.656672,True,-3.123083,7.790683,-2.632508,7.035500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,36.645969,0.002547,-11.656672,True,-3.123083,7.790683,-2.632508,7.035500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000150,37.728565,0.002138,-11.337094,False,-2.708674,7.691035,-2.268131,7.284603,1.191188,...,-0.003768,0.021012,-0.003654,0.021043,-0.003693,0.021435,-0.003868,0.021835,-0.003611,0.022270
3,0.014344,37.728565,0.001876,-11.337094,False,-2.708674,7.691035,-2.268131,7.284603,1.188372,...,-0.006627,0.110317,-0.007127,0.111995,-0.007607,0.114068,-0.007875,0.115976,-0.007781,0.117461
4,0.000132,38.787601,0.001923,-11.057025,False,-2.619153,7.559871,-2.190142,7.034854,1.086010,...,0.001091,0.018090,0.001094,0.018020,0.000964,0.018174,0.000968,0.018386,0.001015,0.018271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0.050575,38.775448,0.001664,-11.282922,False,-8.300492,8.016449,-9.440802,9.261126,1.231618,...,-0.003486,0.090631,-0.003051,0.109357,-0.002420,0.129765,-0.002756,0.140739,-0.003269,0.177529
86,0.001234,39.810261,0.002005,-10.975616,False,-8.340130,7.723401,-8.938865,8.594180,1.099593,...,0.002545,0.028001,0.002044,0.030065,0.001852,0.032554,0.001724,0.033653,0.002306,0.038656
87,0.047706,39.810261,0.001723,-10.975616,False,-8.340130,7.723401,-8.938865,8.594180,1.215896,...,0.003278,0.092518,0.003620,0.108518,0.004722,0.124121,0.005291,0.133901,0.004065,0.167291
88,0.001088,40.821430,0.002622,-10.667827,False,-8.327045,7.528501,-8.020763,7.828473,1.081216,...,-0.001866,0.025538,-0.001749,0.026740,-0.001453,0.027524,-0.001330,0.029135,-0.001464,0.030713


In [88]:
df_res = df_res.assign(file_path=path)
df_res

,sigma0_filt,incidence,nesz,ground_heading,land_flag,uwnd,vwnd,U10,V10,normalized_variance_filt,...,macs_Re_lambda_max=175.0,macs_Im_lambda_max=200.0,macs_Re_lambda_max=200.0,macs_Im_lambda_max=225.0,macs_Re_lambda_max=225.0,macs_Im_lambda_max=250.0,macs_Re_lambda_max=250.0,macs_Im_lambda_max=275.0,macs_Re_lambda_max=275.0,file_path
0,NaN,36.645969,0.002884,-11.656672,True,-3.123083,7.790683,-2.632508,7.035500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/home/datawork-cersat-public/project/sarwave/d...
1,NaN,36.645969,0.002547,-11.656672,True,-3.123083,7.790683,-2.632508,7.035500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/home/datawork-cersat-public/project/sarwave/d...
2,0.000150,37.728565,0.002138,-11.337094,False,-2.708674,7.691035,-2.268131,7.284603,1.191188,...,0.021012,-0.003654,0.021043,-0.003693,0.021435,-0.003868,0.021835,-0.003611,0.022270,/home/datawork-cersat-public/project/sarwave/d...
3,0.014344,37.728565,0.001876,-11.337094,False,-2.708674,7.691035,-2.268131,7.284603,1.188372,...,0.110317,-0.007127,0.111995,-0.007607,0.114068,-0.007875,0.115976,-0.007781,0.117461,/home/datawork-cersat-public/project/sarwave/d...
4,0.000132,38.787601,0.001923,-11.057025,False,-2.619153,7.559871,-2.190142,7.034854,1.086010,...,0.018090,0.001094,0.018020,0.000964,0.018174,0.000968,0.018386,0.001015,0.018271,/home/datawork-cersat-public/project/sarwave/d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0.050575,38.775448,0.001664,-11.282922,False,-8.300492,8.016449,-9.440802,9.261126,1.231618,...,0.090631,-0.003051,0.109357,-0.002420,0.129765,-0.002756,0.140739,-0.003269,0.177529,/home/datawork-cersat-public/project/sarwave/d...
86,0.001234,39.810261,0.002005,-10.975616,False,-8.340130,7.723401,-8.938865,8.594180,1.099593,...,0.028001,0.002044,0.030065,0.001852,0.032554,0.001724,0.033653,0.002306,0.038656,/home/datawork-cersat-public/project/sarwave/d...
87,0.047706,39.810261,0.001723,-10.975616,False,-8.340130,7.723401,-8.938865,8.594180,1.215896,...,0.092518,0.003620,0.108518,0.004722,0.124121,0.005291,0.133901,0.004065,0.167291,/home/datawork-cersat-public/project/sarwave/d...
88,0.001088,40.821430,0.002622,-10.667827,False,-8.327045,7.528501,-8.020763,7.828473,1.081216,...,0.025538,-0.001749,0.026740,-0.001453,0.027524,-0.001330,0.029135,-0.001464,0.030713,/home/datawork-cersat-public/project/sarwave/d...


In [89]:
root_savepath = '/home1/datahome/ljessel/l1c_conversion/'
burst_type = 'intraburst'

safe_name, file_name = path.split(os.sep)[-2:]
savepath = os.path.join(root_savepath, safe_name, burst_type, file_name).replace('.nc', '.csv')

os.makedirs(os.path.dirname(savepath), exist_ok=True)
df_res.to_csv(savepath, index=False)

In [90]:
csv_path = '/home1/datahome/ljessel/l1c_conversion/S1A_IW_XSP__1SDV_20210115T180059_20210115T180125_036152_043D21_9982_B07.SAFE/intraburst/l1c-s1a-iw2-xsp-1sdv-20210115t180059-20210115t180124-036152-043d21-b07.csv' 

pd.read_csv(csv_path)

,sigma0_filt,incidence,nesz,ground_heading,land_flag,uwnd,vwnd,U10,V10,normalized_variance_filt,...,macs_Re_lambda_max=175.0,macs_Im_lambda_max=200.0,macs_Re_lambda_max=200.0,macs_Im_lambda_max=225.0,macs_Re_lambda_max=225.0,macs_Im_lambda_max=250.0,macs_Re_lambda_max=250.0,macs_Im_lambda_max=275.0,macs_Re_lambda_max=275.0,file_path
0,NaN,36.645970,0.002884,-11.656672,True,-3.123083,7.790683,-2.632508,7.035500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/home/datawork-cersat-public/project/sarwave/d...
1,NaN,36.645970,0.002547,-11.656672,True,-3.123083,7.790683,-2.632508,7.035500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/home/datawork-cersat-public/project/sarwave/d...
2,0.000150,37.728565,0.002138,-11.337094,False,-2.708674,7.691035,-2.268130,7.284603,1.191188,...,0.021012,-0.003654,0.021043,-0.003693,0.021435,-0.003868,0.021835,-0.003611,0.022270,/home/datawork-cersat-public/project/sarwave/d...
3,0.014344,37.728565,0.001876,-11.337094,False,-2.708674,7.691035,-2.268130,7.284603,1.188372,...,0.110317,-0.007127,0.111995,-0.007607,0.114068,-0.007875,0.115976,-0.007781,0.117461,/home/datawork-cersat-public/project/sarwave/d...
4,0.000132,38.787600,0.001923,-11.057025,False,-2.619153,7.559871,-2.190142,7.034854,1.086010,...,0.018090,0.001094,0.018020,0.000964,0.018174,0.000968,0.018386,0.001015,0.018271,/home/datawork-cersat-public/project/sarwave/d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0.050575,38.775448,0.001664,-11.282922,False,-8.300492,8.016449,-9.440802,9.261126,1.231618,...,0.090631,-0.003051,0.109357,-0.002420,0.129765,-0.002756,0.140739,-0.003269,0.177529,/home/datawork-cersat-public/project/sarwave/d...
86,0.001234,39.810260,0.002005,-10.975616,False,-8.340130,7.723401,-8.938865,8.594180,1.099593,...,0.028001,0.002044,0.030065,0.001852,0.032554,0.001724,0.033653,0.002306,0.038656,/home/datawork-cersat-public/project/sarwave/d...
87,0.047706,39.810260,0.001723,-10.975616,False,-8.340130,7.723401,-8.938865,8.594180,1.215896,...,0.092518,0.003620,0.108518,0.004722,0.124121,0.005291,0.133901,0.004065,0.167291,/home/datawork-cersat-public/project/sarwave/d...
88,0.001088,40.821430,0.002622,-10.667827,False,-8.327045,7.528501,-8.020763,7.828473,1.081216,...,0.025538,-0.001749,0.026740,-0.001453,0.027524,-0.001330,0.029135,-0.001464,0.030713,/home/datawork-cersat-public/project/sarwave/d...
